In [2]:
import pandas as pd
import numpy as np
from pprint import pprint
import requests

ModuleNotFoundError: No module named 'pandas'

In [ ]:
URL = 'https://discosweb.esoc.esa.int'
token = 'Ijc0ZTVhNDg3LTY1MzAtNGU2YS04ZDk1LTk0MzM5Njc5NTNhOCI.gU9eMy5iu9b0cNWm-1x_ao71xS8'

response = requests.get(
    f'{URL}/api/launches',
    headers={
        'Authorization': f'Bearer {token}',
        'DiscosWeb-Api-Version': '2',
    },
    params={
#         'filter': "contains(name,SpaceX)",
        'sort': '-epoch',
        'page[size]': 100,
        'page[number]': 50

    },
)

doc = response.json()
if response.ok:
    pprint(doc['data'])
else:
    pprint(doc['errors'])

ModuleNotFoundError: No module named 'requests'

In [ ]:
import requests
import csv
import time

# Define API details
URL = 'https://discosweb.esoc.esa.int'
TOKEN = 'ImFkM2IxYzU3LTM3OGQtNGNhNC1iMTZjLTQyZGIxODM4YjMzYSI.E_g3DVwIJOz_oJlalZJ2WLWlXqw'
HEADERS = {
    'Authorization': f'Bearer {TOKEN}',
    'DiscosWeb-Api-Version': '2',
}

# CSV output file
OUTPUT_FILE = 'discos_historical_launch_data.csv'

# Total pages
TOTAL_PAGES = 70
PAGE_SIZE = 100  # Number of entries per page

# Initialize an empty list to store all entries
all_entries = []

# Loop through all pages and fetch data
for page_number in range(1, TOTAL_PAGES + 1):
    print(f"Fetching page {page_number}/{TOTAL_PAGES}...")

    response = requests.get(
        f'{URL}/api/launches',
        headers=HEADERS,
        params={
#             'filter': "contains(objectClass,Debris)",
            'sort': 'epoch',
            'page[size]': PAGE_SIZE,
            'page[number]': page_number,
        },
    )

    if response.ok:
        data = response.json().get('data', [])
        for entry in data:
            # Flatten the attributes dictionary into top-level fields
            flattened_entry = entry.get('attributes', {})
            flattened_entry['id'] = entry.get('id', '')  # Include 'id' field
            all_entries.append(flattened_entry)
    else:
        print(f"Error fetching page {page_number}: {response.status_code}")
        print(response.json().get('errors'))
        break  # Stop the loop if an error occurs

    # Avoid hitting rate limits
    time.sleep(1)

# Write to CSV
if all_entries:
    # Extract column names dynamically
    keys = set()
    for entry in all_entries:
        keys.update(entry.keys())

    keys = list(keys)  # Convert set to list to use as fieldnames

    with open(OUTPUT_FILE, 'w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=keys, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writeheader()

        for entry in all_entries:
            writer.writerow({key: str(entry.get(key, '')) for key in keys})

    print(f"Data successfully saved to {OUTPUT_FILE}")
else:
    print("No data fetched.")
